Урок 5. Парсинг данных. Scrapy. Начало
###Вариант II
Написать программу, которая собирает товары «В тренде» с сайта техники mvideo и складывает данные в БД. Сайт можно выбрать и свой. Главный критерий выбора: динамически загружаемые товары

In [2]:
pip install scrapy

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -U selenium

Note: you may need to restart the kernel to use updated packages.


In [8]:
!pip install webdriver-manager

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.78M/6.78M [00:00<00:00, 7.80MB/s]
C:\Users\ekate\AppData\Local\Temp\ipykernel_7596\1327528027.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from pymongo import MongoClient
from pprint import pprint

In [13]:
client = MongoClient('127.0.0.1', 27017)
db = client['shop']
pr = db.pr

In [14]:
chrome_options = Options()
chrome_options.add_argument("start-maximized")

In [ ]:
driver = webdriver.Chrome(executable_path='./chromedriver', options=chrome_options)
driver.get('https://www.mvideo.ru/')

In [ ]:
part = 1
while part < 6:
    try:
        elem = driver.find_element(By.XPATH, '//span[contains(text(), "В тренде")]')
        break
    except NoSuchElementException:
        driver.execute_script(f"window.scrollTo(0, document.body.scrollHeight / 5 * {part});")
        driver.implicitly_wait(10)
    part += 1

elem = driver.find_element(By.XPATH, '//span[contains(text(), "В тренде")]')
elem.click()

In [ ]:
elem = driver.find_elements(By.CLASS_NAME, 'mvid-carousel-inner')[5]
goods = elem.find_elements(By.CLASS_NAME, 'product-mini-card__name')

for good in goods:
    good_dict = dict()
    name = good.text
    uid = good.find_element(By.XPATH, './/a').get_attribute('href').split('-')[-1]
    link = good.find_element(By.XPATH, './/a').get_attribute('href')
    price = elem.find_elements(By.CLASS_NAME, 'product-mini-card__price')[goods.index(good)].text
    price = int(price.split('\n')[0].replace(' ', ''))
    good_dict['_id'] = uid
    good_dict['name'] = name
    good_dict['link'] = link
    good_dict['price'] = price
    pr.insert_one(good_dict)

for el in pr.find({}):
    pprint(el)